In [3]:
import numpy as np
import tensorflow as tf

x = tf.placeholder(tf.float32, shape=[4,2], name='x')
y = tf.placeholder(tf.float32, shape=[4,1], name='y')

XOR_X = [[0,0],[0,1],[1,0],[1,1]]
XOR_Y = [[0],[1],[1],[0]]

## 使用设定好的weight和bias，参考DL_Chapter.6_Page.179

In [4]:
# 1. with no parameters
with tf.variable_scope("hidden_layer") as scope:
    w1 = tf.constant([[1, 1],[1, 1]], shape=[2, 2], dtype=tf.float32)
    b1 = tf.constant([0, -1], shape=[2,], dtype=tf.float32)
    x_ = tf.nn.relu(tf.add(tf.matmul(x, w1), b1))

with tf.variable_scope("output") as scope:
    w2 = tf.constant([[1], [-2]], shape=[2, 1], dtype=tf.float32)
    b2 = tf.constant([0], shape=[1,], dtype=tf.float32)
    y_ = tf.add(tf.matmul(x_, w2), b2)
    
loss = tf.reduce_mean(tf.square(y - y_))

with tf.Session() as sess:
    x, pred, l = sess.run([x_, y_, loss], feed_dict={x: XOR_X, y: XOR_Y})
    print(f"XOR problem:  mse={l}, y prediction: \n{pred}")    

XOR problem:  mse=0.0, y prediction: 
[[0.]
 [1.]
 [1.]
 [0.]]
